In [4]:
import pandas as pd
import pickle
import numpy as np
import re

In [2]:
df = pd.read_csv("../DataSet/douban_boutique/movie.csv", encoding='gbk')
df.head()

,title,types,score,director,writer,actor,account,country,label,flim_version
0,这个杀手不太冷,"剧情, 动作, 犯罪",9.4,吕克·贝松 Luc Besson,吕克·贝松 Luc Besson,"让·雷诺, 娜塔莉·波特曼, 加里·奥德曼, 丹尼·爱罗, 彼得·阿佩尔, 迈克尔·巴达鲁科...",1088567,法国,1023.25298,2
1,七武士,"动作, 冒险, 剧情",9.2,黑泽明 Akira Kurosawa,"黑泽明 Akira Kurosawa, 小国英雄 Hideo Oguni, 桥本忍 Shin...","三船敏郎, 志村乔, 稻叶义男, 宫口精二, 千秋实, 加东大介, 木村功",91163,日本,83.86996,6
2,蝙蝠侠：黑暗骑士,"剧情, 动作, 科幻, 犯罪, 惊悚",9.1,克里斯托弗·诺兰 Christopher Nolan,"乔纳森·诺兰 Jonathan Nolan, 克里斯托弗·诺兰 Christopher No...","克里斯蒂安·贝尔, 希斯·莱杰, 艾伦·艾克哈特, 迈克尔·凯恩, 玛吉·吉伦哈尔, 加里·...",423487,"美国, 英国",385.37317,1
3,指环王3：王者无敌,"剧情, 动作, 奇幻, 冒险",9.1,彼得·杰克逊 Peter Jackson,"弗兰·威尔士 Fran Walsh, Philippa Boyens, 彼得·杰克逊 Pet...","维果·莫腾森, 伊莱贾·伍德, 西恩·奥斯汀, 丽芙·泰勒, 伊恩·麦克莱恩, 奥兰多·布鲁...",350209,"美国, 新西兰",318.69019,3
4,搏击俱乐部,"剧情, 动作, 悬疑, 惊悚",9.0,大卫·芬奇 David Fincher,"恰克·帕拉尼克 Chuck Palahniuk, Jim Uhls","爱德华·诺顿, 布拉德·皮特, 海伦娜·伯翰·卡特, 扎克·格雷尼尔, 米特·洛夫, 杰瑞德...",465674,"美国, 德国",419.10660,1


In [3]:
type_enum = ["黑色电影","恐怖","悬疑","同性","犯罪","儿童","Reality-TV","喜剧","惊栗","歌舞","科幻","家庭",
             "爱情","Adult","情色","灾难","音乐","运动","鬼怪","历史","冒险","西部","惊悚","奇幻","传记",
             "武侠","古装","战争","动作","剧情","悬念","动画"]

In [12]:
type_director_dict = dict()

for t in type_enum: # 对每种类型
    dir_dict = dict() # 清空[导演-电影数(小于7.7 || 7.7-8.5 || 大于8.5)字典]
    for i in range(df.shape[0]): # 对每一条条目
        typestr = df.iloc[i,1] # 类型字符串
        if re.search(t, typestr) is not None: # 如果不是这个类型
            continue # 就跳过
        dirstr = df.iloc[i,3] # 这个电影的导演
        score = df.iloc[i,2] # 这个电影的得分
        sidx = 0 if score<7 else (1 if score<=8 else 2) # 根据得分知道要在哪个下标位置+1
        # 写入[导演-电影数字典]
        if dirstr in dir_dict:
            dir_dict[dirstr][sidx] += 1
        else:
            dir_dict[dirstr] = [0,0,0]
            dir_dict[dirstr][sidx] += 1
    # 对这个[导演-电影数]字典按电影总数从大到小排序，并只取top4
    sort_list = sorted(dir_dict.items(), key = lambda kv:(sum(kv[1])), reverse=True)[:4]
    # 将四个形如('王晶 Jing Wong', [26, 11, 0])变成形如['王晶 Jing Wong',26, 11, 0]的格式
    result = [[],[],[],[]]
    for j in range(4):
        result[j].append(sort_list[j][0][:9])
        result[j].extend(sort_list[j][1])
    # 写入目标字典该类型的位置
    type_director_dict[t] = result

type_director_dict

{'黑色电影': [['王晶 Jing Wong', 26, 11, 0],
  ['杜琪峰 Johnnie To', 10, 16, 2],
  ['徐克 Hark Tsui', 13, 12, 1],
  ['邱礼涛 Herman Yau', 23, 3, 0]],
 '恐怖': [['王晶 Jing Wong', 26, 11, 0],
  ['杜琪峰 Johnnie To', 10, 16, 2],
  ['史蒂文·斯皮尔伯格 Steven Spielberg', 2, 12, 11],
  ['徐克 Hark Tsui', 11, 12, 1]],
 '悬疑': [['王晶 Jing Wong', 26, 11, 0],
  ['杜琪峰 Johnnie To', 10, 15, 2],
  ['史蒂文·斯皮尔伯格 Steven Spielberg', 2, 10, 10],
  ['徐克 Hark Tsui', 10, 11, 1]],
 '同性': [['王晶 Jing Wong', 26, 11, 0],
  ['杜琪峰 Johnnie To', 10, 16, 2],
  ['徐克 Hark Tsui', 13, 12, 1],
  ['邱礼涛 Herman Yau', 23, 3, 0]],
 '犯罪': [['王晶 Jing Wong', 25, 10, 0],
  ['徐克 Hark Tsui', 13, 10, 1],
  ['史蒂文·斯皮尔伯格 Steven Spielberg', 2, 11, 10],
  ['Woody Allen', 0, 11, 8]],
 '儿童': [['王晶 Jing Wong', 26, 11, 0],
  ['杜琪峰 Johnnie To', 10, 16, 2],
  ['徐克 Hark Tsui', 13, 12, 1],
  ['邱礼涛 Herman Yau', 23, 3, 0]],
 'Reality-TV': [['王晶 Jing Wong', 26, 11, 0],
  ['杜琪峰 Johnnie To', 10, 16, 2],
  ['徐克 Hark Tsui', 13, 12, 1],
  ['邱礼涛 Herman Yau', 23, 3, 0]],
 '喜剧': [['史蒂文·斯皮尔

In [13]:
with open('../DataSet/douban_boutique/type_director_dict.pkl','wb') as f:
    pickle.dump(type_director_dict,f)